In [2]:
#SIFT+KMean+VLAD+PCA
#https://github.com/Lithogenous/VLAD-SIFT-python/blob/master/vlad_raw.py
import sys
import os
import math
import random
import heapq 
import numpy as np
import pandas as pd
from PIL import Image
import psycopg2 as ps
from io import StringIO,BytesIO 
import pywt
import cv2
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from functools import reduce

In [2]:
conn = ps.connect(host="127.0.0.1", user="postgres", password="postgres", database="asoct")#connect postgresql 
if conn is not None:
    cur = conn.cursor()
    #id SERIAL primary key
    command = "create table cataract_sift (name text NOT NULL, des float8[] NOT NULL);"
    cur.execute(command)
    conn.commit()# commit the changes
    cur.close()# close communication with the PostgreSQL database server
conn.close()

In [5]:
# SIFT
def sift_extractor(file_path):
    '''
    Description: extract \emph{sift} feature from given image
    Input: file_path - image path
    Output: des - a list of descriptors of all the keypoint from the image
    '''
    img = cv2.imread(file_path)
    gray= cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    sift = cv2.xfeatures2d.SIFT_create()
    _,des = sift.detectAndCompute(gray,None) 

    return des

conn = ps.connect(host="127.0.0.1", user="postgres", password="postgres", database="asoct")#connect postgresql 
if conn is not None:
    cur = conn.cursor()
    #Extract features with SIFT
    image_dir = '/data/fjsdata/ASOCT/Cataract/C_8bit_Crop_New' #the path of images
    num_img = 0 #count
    for fname in sorted(os.listdir(image_dir)):
        num_img = num_img+1
        sys.stdout.write('\r{} / {} '.format(num_img,40066))
        sys.stdout.flush()
        if fname.endswith(".jpg"):
            try:
                des = sift_extractor(os.path.join(image_dir, fname))
                command = "insert into cataract_sift (name, des) values(%s, %s);"
                params = (fname, des.tolist())
                cur.execute(command, params)
                conn.commit()# commit the changes 
            except:
                print(fname)
                print("extract feature error")              
                continue
    cur.close()# close communication with the PostgreSQL database server
conn.close()

40066 / 40066 

In [3]:
#KMeans
def get_cluster_center(des_set, K):
    '''
    Description: cluter using a default setting
    Input: des_set - cluster data
                 K - the number of cluster center
    Output: laber  - a np array of the nearest center for each cluster data
            center - a np array of the K cluster center
    '''
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 0.01)
    des_set = np.float32(des_set)
    ret, label, center = cv2.kmeans(des_set, K, None, criteria, 3, cv2.KMEANS_RANDOM_CENTERS)
    return label, center

def get_codebook(all_des, K):
    '''
    Description: train the codebook from all of the descriptors
    Input: all_des - training data for the codebook
                 K - the column of the codebook
    '''
    label, center = get_cluster_center(all_des, K)
    return label, center

#query and show image
conn = ps.connect(host="127.0.0.1", user="postgres", password="postgres", database="asoct")#connect postgresql 
if conn is not None:
    cur = conn.cursor() 
    command = "select name,des from cataract_sift limit 100;"
    cur.execute(command) 
    all_des = np.empty(shape=[0, 128])
    image_des_len = []
    image_name = []
    for name, des in cur.fetchall():
        all_des = np.concatenate([all_des, np.array(des)])
        image_des_len.append(len(des))
        image_name.append(name)
    cur.close()
conn.close()
kplabel, codebook = get_codebook(all_des, K=32)#clusters 32

In [4]:
#vlad
def get_vlad_base_pca(all_des, img_des_len, NNlabel, codebook):
    cursor = 0
    vlad_base = []
    for eachImage in img_des_len:
        descrips = all_des[cursor : cursor + eachImage]
        centriods_id = NNlabel[cursor : cursor + eachImage]
        centriods = codebook[centriods_id]
    
        vlad = np.zeros(shape=[COLUMNOFCODEBOOK, DESDIM])
        for eachDes in range(eachImage):
            vlad[centriods_id[eachDes]] = vlad[centriods_id[eachDes]] + descrips[eachDes] - centriods[eachDes]
        cursor += eachImage

        vlad_base.append(vlad.reshape(COLUMNOFCODEBOOK * DESDIM, -1))
    
    vlad_base_pca = np.array(vlad_base)
    vlad_base_pca = vlad_base_pca.reshape(-1, DESDIM * COLUMNOFCODEBOOK)
    sklearn_pca = sklearnPCA(n_components=PCAD)
    sklearn_transf = sklearn_pca.fit_transform(vlad_base_pca)
    sklearn_transf_norm = sklearn_transf.copy()
    for each, each_norm in zip(sklearn_transf, sklearn_transf_norm):
        cv2.normalize(each, each_norm, 1.0, 0.0, cv2.NORM_L2)
    return sklearn_transf_norm, sklearn_pca

def get_vlad_base(all_des, img_des_len, NNlabel,  codebook):
    '''
    Description: get all images vlad vector 
    '''
    cursor = 0
    vlad_base = []
    for eachImage in img_des_len:
        descrips = all_des[cursor : cursor + eachImage]
        centriods_id = NNlabel[cursor : cursor + eachImage]
        centriods = codebook[centriods_id]
    
        vlad = np.zeros(shape=[32, 128])
        for eachDes in range(eachImage):
            vlad[centriods_id[eachDes]] = vlad[centriods_id[eachDes]] + descrips[eachDes] - centriods[eachDes]
        cursor += eachImage
    
        vlad_norm = vlad.copy()
        cv2.normalize(vlad, vlad_norm, 1.0, 0.0, cv2.NORM_L2)
        vlad_base.append(vlad_norm.reshape(32 * 128, -1))

    return vlad_base

vlad_base = get_vlad_base(all_des, image_des_len, kplabel, codebook) #vlad only

In [8]:
#test and evaluation
def sift_extractor(file_path):
    '''
    Description: extract \emph{sift} feature from given image
    Input: file_path - image path
    Output: des - a list of descriptors of all the keypoint from the image
    '''
    img = cv2.imread(file_path)
    gray= cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    sift = cv2.xfeatures2d.SIFT_create()
    _,des = sift.detectAndCompute(gray,None) 

    return des

def get_pic_vlad_pca(pic, des_size, codebook, sklearn_pca):
    vlad = np.zeros(shape=[COLUMNOFCODEBOOK, DESDIM])
    for eachDes in range(des_size):
        des = pic[eachDes]
        min_dist = 1000000000.0
        ind = 0
        for i in range(COLUMNOFCODEBOOK):
            dist = cal_vec_dist(des, codebook[i])
            if dist < min_dist:
                min_dist = dist
                ind = i
        vlad[ind] = vlad[ind] + des - codebook[ind]
    
    vlad = vlad.reshape(-1, COLUMNOFCODEBOOK * DESDIM)
    sklearn_transf = sklearn_pca.transform(vlad)
    sklearn_transf_norm = sklearn_transf.copy()
    cv2.normalize(sklearn_transf, sklearn_transf_norm, 1.0, 0.0, cv2.NORM_L2)
    
    return sklearn_transf_norm

def cal_vec_dist(vec1, vec2):
    '''
    Description: calculate the Euclidean Distance of two vectors
    '''
    return np.linalg.norm(vec1 - vec2)

def get_pic_vlad(pic, des_size, codebook):
    '''
    Description: get the vlad vector of each image
    '''
    vlad = np.zeros(shape=[32, 128])
    for eachDes in range(des_size):
        des = pic[eachDes]
        min_dist = 1000000000.0
        ind = 0
        for i in range(32):
            dist = cal_vec_dist(des, codebook[i])
            if dist < min_dist:
                min_dist = dist
                ind = i
        vlad[ind] = vlad[ind] + des - codebook[ind]
    
    vlad_norm = vlad.copy()
    cv2.normalize(vlad, vlad_norm, 1.0, 0.0, cv2.NORM_L2)
    vlad_norm = vlad_norm.reshape(32 * 128, -1)
    
    return vlad_norm

image_dir = '/data/fjsdata/ASOCT/Cataract/C_8bit_Crop_New' #the path of images
data = pd.read_csv("/data/fjsdata/ASOCT/Cataract/CBIR_Cataract.csv" , sep=',')#load dataset
print('Dataset Statistics: Rows = %d, Cols = %d' % (data.shape[0], data.shape[1]))
name_list = data['A'].tolist()

def hit(name, smname):
    if smname not in name_list:#no label
        return 0
    else:
        lv_name = data.loc[data.A==name,'D'].values[0]
        lv_smname = data.loc[data.A==smname,'D'].values[0]
        if ( lv_name== lv_smname): #same level,hit
            return 1
        return 0  
    
for topk in [5,10,15]:
    HR =[] #Hit ratio 
    MAP =[] #mean average precision
    for tname in random.sample(name_list,100):
        tdes = sift_extractor(os.path.join(image_dir, tname))
        tvlad = get_pic_vlad(tdes, len(tdes), codebook)
        map_item_score = {}
        for eachpic in range(len(image_name)):
            dist = cal_vec_dist(tvlad, vlad_base[eachpic])
            map_item_score[image_name[eachpic]] = dist
        #performance    
        pos_len = 0
        rank_len = 0
        rank_map = []
        #choose the topk nearest images of the given image 
        ranklist = heapq.nlargest(topk, map_item_score, key=map_item_score.get)
        for smname in ranklist:
            ret = hit(tname,smname)
            HR.append(ret)
            rank_len=rank_len+1
            if ret==1: 
                pos_len = pos_len +1
                rank_map.append(pos_len/rank_len) 
            else: 
                rank_map.append(0)
        MAP.append(np.mean(rank_map))  
    print("HR@{}={:.6f}, MAP@{}={:.6f}".format(topk,np.mean(HR),topk,np.mean(MAP)))  

Dataset Statistics: Rows = 31998, Cols = 4
HR@5=0.148000, MAP@5=0.145400
HR@10=0.094000, MAP@10=0.091168
HR@15=0.150000, MAP@15=0.147191
